In [1]:
import geopandas as gpd
import pickle
import os
from pathlib import Path


In [2]:
export_fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot/gps_traces'

In [3]:
with (export_fp/'raw_coords.pkl').open('rb') as fh:
    raw_coords_dict, raw_trips_df = pickle.load(fh)

In [5]:
with (export_fp/'cleaned_traces.pkl').open('rb') as fh:
    coords_dict, trips_df = pickle.load(fh)

In [7]:
from datetime import timedelta

In [12]:
tripid = trips_df.loc[(trips_df['duration']>timedelta(minutes=20)) & (trips_df['duration']<timedelta(minutes=40)),'tripid'].sample(1).item()

In [26]:
trips_df.columns

Index(['tripid', 'initial_start_time', 'initial_end_time', 'initial_duration',
       'initial_total_points', 'initial_avg_accuracy', 'tot_points',
       'duration', 'min_time_difference', 'max_time_difference',
       'mean_time_difference', 'min_distance_ft', 'max_distance_ft',
       'avg_distance_ft', 'total_distance_ft', 'max_speed_mph',
       'min_speed_mph', 'avg_speed_mph', 'total_points'],
      dtype='object')

In [31]:
import folium
from folium import plugins
import geopandas as gpd

raw_gdf = raw_coords_dict[tripid]
cleaned_gdf = coords_dict[tripid]

# Define the EPSG code for the projection
projection_epsg = 2240

# Reproject GeoDataFrames to EPSG:4326 for Folium
raw_gdf = raw_gdf.to_crs(epsg=4326)
cleaned_gdf = cleaned_gdf.to_crs(epsg=4326)

# Create a Folium map with the specified projection
map_center = [raw_gdf.geometry.y.mean(), raw_gdf.geometry.x.mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Add raw points to the map with black color and transparency
for idx, row in raw_gdf.iterrows():
    folium.CircleMarker(location=[row['geometry'].y, row['geometry'].x],
                        radius=5,
                        color='black',
                        fill=True,
                        fill_color='black',
                        fill_opacity=0.5,
                        popup='Raw Point').add_to(mymap)

# Add cleaned points to the map with triangles for the first and last points
for idx, row in cleaned_gdf.iterrows():
    tripid = row['tripid']
    if idx == cleaned_gdf.index[0]:
        fill_color = 'green'
        marker_icon = 'play'
        title = f"Start - Trip ID: {tripid}"
    elif idx == cleaned_gdf.index[-1]:
        fill_color = 'darkred'  # Make the last point darker red
        marker_icon = 'stop'
        title = f"End - Trip ID: {tripid}"
    else:
        fill_color = 'red'
        marker_icon = None  # No icon for intermediate points
        title = f"Intermediate - Trip ID: {tripid}"
    
    folium.CircleMarker(location=[row['geometry'].y, row['geometry'].x],
                        radius=5,
                        color=fill_color,
                        fill=True,
                        fill_color=fill_color,
                        fill_opacity=0.5,
                        title=title).add_to(mymap)
    
    if marker_icon is not None:
        marker_icon = folium.Icon(color=fill_color, icon=marker_icon, prefix='fa')
        folium.Marker(location=[row['geometry'].y, row['geometry'].x],
                      icon=marker_icon,
                      title=title).add_to(mymap)

# Calculate trip length and duration (replace with your actual calculations)
trip_length = trips_df.loc[trips_df['tripid']==tripid,'total_distance_ft'].item() / 5280  # replace with actual trip length
trip_duration = trips_df.loc[trips_df['tripid']==tripid,'duration'].item()  # replace with actual trip duration

# Add legend with statistics
legend_html = f'''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 150px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;
                 opacity: 0.9;">
     &nbsp; <b>Tripid: {tripid}</b> <br>
     &nbsp; Raw Point &nbsp; <i class="fa fa-circle" style="color:black"></i><br>
     &nbsp; Start Point &nbsp; <i class="fa fa-play" style="color:green"></i><br>
     &nbsp; End Point &nbsp; <i class="fa fa-stop" style="color:red"></i><br>
     &nbsp; Trip Length: {trip_length:.2f} mi <br>
     &nbsp; Trip Duration: {trip_duration}
      </div>
     '''
mymap.get_root().html.add_child(folium.Element(legend_html))

# Display the map in the notebook
#mymap.save("map_comparison.html")
mymap
